In [ ]:
import math
import gzip
import pickle
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.autograd import Variable
from torch.distributions import Normal

from s2cnn.nn.soft.so3_conv import SO3Convolution
from s2cnn.nn.soft.s2_conv import S2Convolution
from s2cnn.nn.soft.so3_integrate import so3_integrate
from s2cnn.ops.so3_localft import near_identity_grid as so3_near_identity_grid
from s2cnn.ops.s2_localft import near_identity_grid as s2_near_identity_grid

import matplotlib.pyplot as plt

from pytorch_util import load_mnist_data
from vae import NS2VAE

In [1]:
import numpy as np

In [2]:
a = np.array([1,2,3])
b = np.array([1,8,4])

In [5]:
0 + a + b

array([ 2, 10,  7])

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', '-g', type=int, default=0, help='')
parser.add_argument('--epochs', '-e', type=int, default=1, help='')
parser.add_argument('--batch_dim', '-bd', type=int, default=32, help='')
parser.add_argument('--learning_rate', '-lr', type=float, default=1e-3, help='')

FLAGS, unparsed = parser.parse_known_args()

In [ ]:
train_loader, test_loader, train_dataset, _ = load_mnist_data('../mnist_example/s2_mnist.gz', FLAGS.batch_dim)

torch.cuda.set_device(FLAGS.gpu)

In [ ]:
vae = NS2VAE(z_dim = 30)

print("#params", sum([x.numel() for x in vae.parameters()]))

if torch.cuda.is_available():
    vae.cuda(FLAGS.gpu)

optimizer = torch.optim.Adam(vae.parameters(), lr=FLAGS.learning_rate)

In [ ]:
for epoch in range(FLAGS.epochs):
    print ('\nepoch: %d' % epoch)
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)/255
        labels = Variable(labels)

        if torch.cuda.is_available():
            images = images.cuda(FLAGS.gpu)
            labels = labels.cuda(FLAGS.gpu)
        
        rec, kl = vae.elbo(images)
        elbo = rec #+ kl.mean()
        elbo.backward()

        optimizer.step()
        optimizer.zero_grad()
        
        print('\r it: %d/%d \t elbo: %4.3f' % 
              (i, int(len(train_dataset) / FLAGS.batch_dim), elbo.cpu().data.numpy()), end='')

In [ ]:
# plt.imshow(F.sigmoid(vae(images)[0,np.random.randint(0,31),0]).cpu().data.numpy(), cmap="gray")
# plt.show()